In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [6]:
# Load landmark data
landmarks_df = pd.read_csv('../data/HaGRID/modified/csv/train_label_balanced.csv')
landmarks_df.head()

,image_path,label,handedness,x_0,y_0,z_0,x_1,y_1,z_1,x_2,...,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20
0,MODIFIED/call/c59f650f-5f8a-47e8-b948-f58c7b7a...,call,Right,0.754290,0.386637,-4.098337e-07,0.749103,0.344792,0.001934,0.726038,...,-0.035032,0.595950,0.367655,-0.041725,0.564724,0.370839,-0.039518,0.541878,0.370556,-0.037125
1,MODIFIED/call/0e6e38df-275c-4b54-bae2-cbadfe7d...,call,Left,0.402540,0.481069,-1.031420e-07,0.410768,0.462422,-0.001044,0.423757,...,-0.011130,0.453464,0.498888,-0.013328,0.463487,0.505605,-0.012871,0.472385,0.510082,-0.012735
2,MODIFIED/call/b2432f41-2b8e-409c-91ff-7b777fb1...,call,Right,0.674610,0.532156,-1.900930e-07,0.664562,0.495894,-0.000400,0.645847,...,-0.021909,0.598007,0.527357,-0.026023,0.591233,0.529411,-0.024737,0.587414,0.526515,-0.023360
3,MODIFIED/call/7192505e-d304-4f86-9f4b-8a537767...,call,Right,0.656416,0.334383,-4.484448e-07,0.605983,0.291761,0.001999,0.536142,...,-0.083188,0.489501,0.410172,-0.093585,0.450586,0.429055,-0.093920,0.418757,0.441578,-0.096475
4,MODIFIED/call/de9a7ee3-8baa-4289-8970-a7b796a7...,call,Left,0.340446,0.560608,6.133277e-08,0.332125,0.527111,-0.012109,0.345604,...,-0.021573,0.479623,0.556234,-0.027739,0.507211,0.556837,-0.026317,0.530148,0.557762,-0.024677


In [7]:
# Prepare features and labels
X = landmarks_df.drop(columns=['image_path', 'handedness', 'label']).values
y = landmarks_df['label'].values

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [8]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape data for LSTM input
X_train = X_train.reshape((X_train.shape[0], 21, 3))
X_test = X_test.reshape((X_test.shape[0], 21, 3))

In [9]:
# Define an LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(21, 3)))
model.add(Dense(len(le.classes_), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2024-07-17 13:03:55.207735: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-17 13:03:55.356999: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-17 13:03:55.361778: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [11]:
# Train the model
model.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test))

model_base_dir = '../models/HaGRID/LSTM/'

# Save the model and label encoder
model.save(model_base_dir + 'model.keras')
np.save(model_base_dir + 'label_encoder_classes.npy', le.classes_)

Epoch 1/25
  51/6696 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.9678 - loss: 0.1129 

2024-07-17 13:14:54.764343: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 53994024 exceeds 10% of free system memory.
2024-07-17 13:14:54.824059: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 53994024 exceeds 10% of free system memory.


6696/6696 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9727 - loss: 0.0873 - val_accuracy: 0.9715 - val_loss: 0.0946
Epoch 2/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9719 - loss: 0.0902 - val_accuracy: 0.9705 - val_loss: 0.0975
Epoch 3/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9732 - loss: 0.0867 - val_accuracy: 0.9716 - val_loss: 0.0925
Epoch 4/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9729 - loss: 0.0862 - val_accuracy: 0.9714 - val_loss: 0.0929
Epoch 5/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9728 - loss: 0.0866 - val_accuracy: 0.9724 - val_loss: 0.0906
Epoch 6/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9733 - loss: 0.0844 - val_accuracy: 0.9729 - val_loss: 0.0881
Epoch 7/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9736 - loss: 0.0842 - val_accuracy: 0.9726 - val_loss: 0.0894
Epoch 8/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9731 - loss: 0.0848 - val